In [4]:
from model.module.path_header import *

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split

import os

In [5]:
TRAIN_NAME = '0906-21-train-v2.csv'
TEST1_NAME = '0906-21-test1-v2.csv'
TEST2_NAME = '0906-21-test2-v2.csv'
LABEL_NAME = 'train_label.csv'

TRAIN_PATH = os.path.join(PREPROCESS_DIR, TRAIN_NAME)
TEST1_PATH = os.path.join(PREPROCESS_DIR, TEST1_NAME)
TEST2_PATH = os.path.join(PREPROCESS_DIR, TEST2_NAME)
LABEL_PATH = os.path.join(PREPROCESS_DIR, LABEL_NAME)

train = pd.read_csv(TRAIN_PATH)
test1 = pd.read_csv(TEST1_PATH)
test2 = pd.read_csv(TEST2_PATH)
label = pd.read_csv(LABEL_PATH)

In [87]:
tra = train.copy()
te1 = test1.copy()
te2 = test2.copy()
lab = label.copy()

In [88]:
tra_k = pd.read_csv(os.path.join(PREPROCESS_DIR, 'train-knn-clustering.csv'))
te1_k = pd.read_csv(os.path.join(PREPROCESS_DIR, 'test1-knn-clustering.csv'))
te2_k = pd.read_csv(os.path.join(PREPROCESS_DIR, 'test2-knn-clustering.csv'))

In [89]:
tra = pd.merge(tra, tra_k, on='acc_id')
te1 = pd.merge(te1, te1_k, on='acc_id')
te2 = pd.merge(te2, te2_k, on='acc_id')

In [90]:
tra_acc_id = np.unique(tra.acc_id)
te1_acc_id = np.unique(te1.acc_id)
te2_acc_id = np.unique(te2.acc_id)

In [91]:
tra['isPlay'] = tra['a_playtime'].transform(lambda x: 1 if x!=0 else 0)
te1['isPlay'] = tra['a_playtime'].transform(lambda x: 1 if x!=0 else 0)
te2['isPlay'] = tra['a_playtime'].transform(lambda x: 1 if x!=0 else 0)

In [92]:
tra.drop(['acc_id', 'day'], axis=1, inplace=True)
te1.drop(['acc_id', 'day'], axis=1, inplace=True)
te2.drop(['acc_id', 'day'], axis=1, inplace=True)

In [93]:
tra = np.asarray(tra)
te1 = np.asarray(te1)
te2 = np.asarray(te2)

tra = tra.reshape(40000,28,-1)
te1 = te1.reshape(20000,28,-1)
te2 = te2.reshape(20000,28,-1)

In [94]:
tra.shape

(40000, 28, 74)

In [95]:
tra[0].shape

(28, 74)

In [106]:
time_length = 7
data7 = []
for data in tra:
    tmp = []
    for i in range(tra.shape[1]-time_length+1): ## range(22)
        tmp.append(data[i:i+time_length, :])
    data7.append(tmp)
data7 = np.asarray(data7)
data7.shape

(40000, 22, 7, 74)

In [107]:
input_X = data7[:, :-1, :, :]
output = data7[:, 1:, :, [-2]] # isSurvival

In [109]:
print(input_X.shape)
print(output.shape)

(40000, 21, 7, 74)
(40000, 21, 7, 1)


In [115]:
input_X = input_X.reshape(-1,7,74)
output = output.reshape(-1,7,1)

In [116]:
print(input_X.shape)
print(output.shape)

(840000, 7, 74)
(840000, 7, 1)


In [146]:
time_length = 7
n_inputs = input_X.shape[2]
n_outputs = output.shape[2]
n_neurons = 8
n_epochs = 10
batch_size = 40000
num_acc = input_X.shape[0]
n_batches = num_acc//batch_size
learning_rate = 0.1

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, time_length, n_inputs])
y = tf.placeholder(tf.float32, shape=[None, time_length, n_outputs])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

outputs = outputs[:, -1, :] # 7 

w = tf.get_variable('w', shape=[n_neurons, n_outputs], initializer=tf.contrib.layers.xavier_initializer())
b = tf.get_variable('b', shape=[n_outputs], initializer=tf.contrib.layers.xavier_initializer())

hf = tf.matmul(outputs, w)+b

loss = tf.reduce_mean(tf.square(hf - y))
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

In [148]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        _, _loss = sess.run([training_op, loss], feed_dict={X: input_X, y: output})
            
        if epoch % 100 == 0:
            val_loss = sess.run(loss, feed_dict={X: X_val, y: y_val})
            print(f"Epoch: {epoch} \t| MSE: {_loss:.8f} \t| Validation Loss: {val_loss:.8f}")
            
    test_index = np.random.randint(0, int(num_accounts * val_size), 15)
    y_pred = sess.run(hf, feed_dict={X: X_val[test_index]})
    y_true = y_val[test_index]
    
    for p, t in zip(y_pred, y_true):
        print("=" * 40)
        for _p, _t in zip(p, t):
            print(f"Predicted: {_p[0]:.8f}, \tActual: {_t[0]:.8f}")
            
    sequence_data = [sess.run(hf, feed_dict={X:X_test1})]      

InvalidArgumentError: Incompatible shapes: [840000,1] vs. [840000,7,1]
	 [[node gradients/sub_grad/BroadcastGradientArgs (defined at <ipython-input-146-6e8185f90162>:28) ]]

Original stack trace for 'gradients/sub_grad/BroadcastGradientArgs':
  File "C:\Users\MAIN\Anaconda3\envs\kaggle\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\MAIN\Anaconda3\envs\kaggle\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\MAIN\Anaconda3\envs\kaggle\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\MAIN\Anaconda3\envs\kaggle\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\MAIN\Anaconda3\envs\kaggle\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\MAIN\Anaconda3\envs\kaggle\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.a